In [1]:
# pip install fastai==1.0.61
from enum import Enum
import os
import fastai
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.utils import midifile
from mido import MidiFile
import numpy as np
import tensorflow as tf

In [ ]:
#)_______________________________dont run this

In [ ]:

#item = MusicItem.from_file('wii.mid', MusicVocab.create())
#print(item.to_text())
#print(len(item))
#print(item.to_tensor())
path = "C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\6.mid"
print(midi2npenc(path)) #throws warning
embedding = midi2idxenc(path, MusicVocab.create())
print(embedding)
print(len(embedding))

#tempo exploration with mido- what kind of tempo do these tracks have
mid = MidiFile(path)
for msg in mid:
    if msg.type == 'set_tempo':
        print('Tempo set to', msg.tempo)

In [ ]:
path_list = []
path = "C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi"
for i in os.listdir(path):
    newpath = path + "\\" + i
    print(newpath)
    path_list.append(newpath)

In [ ]:
# i have to filter out the things that will cause a keyerror for no reason
vocab = MusicVocab.create()
good = []
for i in range(len(path_list)):
    try:
        print(path_list[i])
        enc = midi2idxenc(path_list[i], vocab)
        print(enc)
        print(len(enc))
        good.append(path_list[i])
    except:
        print("error")

print(len(good))

In [ ]:
#save array as text file
with open("paths.txt", "w") as f:
    for s in good:
        f.write(s +"\n")

In [ ]:
# ______________________________________

In [3]:
path_list = []
with open("paths.txt", "r") as f:
  for line in f:
    path_list.append(line.strip())

In [4]:
print(path_list)

['C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\10.mid', 'C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\100.mid', 'C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\1000.mid', 'C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\10000.mid', 'C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\10001.mid', 'C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\10002.mid', 'C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\10003.mid', 'C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\10004.mid', 'C:\\Users\\drago\\OneDrive\\Documents\\MOVE TO NEW PC\\College\\fall 2022\\FIRE\\adl-piano-midi\\10005.mid', 'C:\\Users\\dra

In [3]:
vocab = MusicVocab.create()
lst = []
# for some reason the first midi doesn't work
for i in path_list:
    print(i)
    enc = midi2idxenc(i, vocab)
    print(enc)
    print(len(enc))
    lst.append(enc)
np.save("data.npy", lst)
#1000 files were only 12.8 mb. my memory has a GB of space for sure, so I don't think i need a data generator.

NameError: name 'path_list' is not defined

In [8]:
# TODO: NORMALIZE DATA!!!!!!!!!!!!!
data = np.load("data.npy", allow_pickle = True)

data /= 311 # number of things in dict
print(data)
np.save("data_normalized.npy", data)       

[array([0.      , 0.003215, 0.025723, 0.491961, ..., 0.18328 , 0.466238, 0.167203, 0.466238])
 array([0.      , 0.003215, 0.282958, 0.44373 , ..., 0.170418, 0.491961, 0.14791 , 0.491961])
 array([0.      , 0.003215, 0.257235, 0.466238, ..., 0.21865 , 0.491961, 0.205788, 0.491961])
 array([0.      , 0.003215, 0.025723, 0.491961, ..., 0.025723, 0.446945, 0.157556, 0.44373 ]) ...
 array([0.      , 0.003215, 0.282958, 0.453376, ..., 0.221865, 0.453376, 0.209003, 0.453376])
 array([0.      , 0.003215, 0.025723, 0.491961, ..., 0.199357, 0.446945, 0.189711, 0.446945])
 array([0.      , 0.003215, 0.025723, 0.491961, ..., 0.22508 , 0.44373 , 0.209003, 0.44373 ])
 array([0.      , 0.003215, 0.025723, 1.160772, ..., 0.237942, 0.44373 , 0.221865, 0.44373 ])]


In [4]:
data = np.load("data_normalized.npy", allow_pickle = True)
len(data)

11045

In [2]:
data = np.load("data_normalized.npy", allow_pickle = True)
array = data[0]
print(array)
i = tf.keras.preprocessing.timeseries_dataset_from_array(
    array,
    None,
    101, #sequence length
    sequence_stride=1,
    sampling_rate=1,
    batch_size=len(array), #will not reach the batch size
    shuffle=False,
    seed=None,
    start_index=None,
    end_index=None,
)
#there's only one e because the batch size is so large
for e in i:
    x = np.zeros((len(e), 100))
    y = np.zeros((len(e), 1))
    # iterate over every tensor and arrange it into x and y pairs
    for j in range(len(e)):
        x[j] = e[j][:100]
        y[j] = e[j][-1]
        
# now we have 2 np arrays of what we want
print(x.shape)
print(y.shape)

[  0   1   8 153 ...  57 145  52 145]
(4008, 100)
(4008, 1)


In [9]:

start = 0
index = 0
stop = 11044
while start <= 11044:
    x = None
    y = None
    for array in data[start:min(start+50, 11045)]:
        print(len(array))
        i = tf.keras.preprocessing.timeseries_dataset_from_array(
            array,
            None,
            102, #sequence length
            sequence_stride=2,
            sampling_rate=1,
            batch_size=len(array), #will not reach the batch size
            shuffle=False,
            seed=None,
            start_index=None,
            end_index=None,
        )
        for e in i:
            new_x = np.zeros((len(e), 100))
            new_y = np.zeros((len(e), 2))
            # iterate over every tensor and arrange it into x and y pairs
            for j in range(len(e)):
                new_x[j] = e[j][:100]
                new_y[j] = e[j][-2:]
            if x is None and y is None:
                x = new_x
                y = new_y
            else:
                x = np.concatenate((x, new_x))
                y = np.concatenate((y, new_y))
    print(x.shape)
    print(y.shape)
    np.save("x_" + str(index) + ".npy", x)
    np.save("y_" + str(index) + ".npy", y)
    index += 1
    start = 50 * index
    
# 3 arrays take 15 secs, so 11000 arrays should take 55000 secs = 15 HOURS 
# update: new process, 3 arrays take 8-9 secs. 

4108
5808
1006
3364
1622
558
1372
594
620
2436
5820
2858
4764
3374
1344
990
3930
2842
602
4106
502
28612
3254
1350
414
3164
4284
3876
522
3826
7796
2012
786
950
584
4110
4350
3166
1778
6124
2438
1200
4124
3002
2492
2726
1650
2852
3780
2638
(77740, 100)
(77740, 2)
702
3414
1102
2932
3494
3768
942
2638
1632
2466
9588
2050
896
228
5628
510
2874
3526
5432
16736
3296
4934
4764
1082
1208
3272
4070
4396
1650
4082
1044
4108
4812
3720
7046
5352
3504
3774
782
2434
6022
5104
1130
2108
806
1672
440
180
2248
970
(77784, 100)
(77784, 2)
2242
5276
5938
358
2384
946
2522
1214
2134
5066
4888
5728
1802
7180
266
5144
12102
3174
2630
3628
1776
3178
6102
4272
13674
1050
4008
13354
1896
5330
4480
666
11844
7542
7712
4144
310
6058
7450
2800
8700
1282
1550
1154
778
1262
1164
3258
6710
2558
(102842, 100)
(102842, 2)
7414
704
1426
2772
940
2474
2224
7136
550
6846
3202
518
3642
1762
1810
1424
3350
5124
5384
2814
7070
5520
4830
1608
1616
3030
1982
2024
2860
1172
2366
2928
3016
272
3008
1298
1280
1366
12426
4232
7

1760
1930
4014
(104017, 100)
(104017, 2)
3298
2488
4894
10294
4008
5974
22864
6542
3764
2140
2772
1224
9420
2576
5430
6282
4142
4652
5740
10036
10826
7126
3916
6028
2204
5058
4480
5972
3574
6578
6194
4204
4680
14366
6580
6560
5166
5716
7646
5368
4350
5362
5194
6590
4374
1556
2088
6350
3658
5426
(140365, 100)
(140365, 2)
3978
5200
5130
9032
3788
10576
2744
6550
8850
13502
5548
6836
5452
16442
1874
3454
1936
1826
4346
4242
3376
2580
3988
6542
3182
3116
444
2042
4536
546
7048
14582
9446
2348
4730
3424
3778
2544
5896
966
7014
15418
9512
3786
2074
1454
2336
2218
4162
1314
(125354, 100)
(125354, 2)
11850
8518
658
1224
2578
1430
2550
1786
1152
1462
428
9276
756
1474
1532
2538
1408
660
1432
2416
1088
1072
4458
926
1968
1184
3696
1432
1576
1752
1152
1356
920
2222
2302
894
2132
2500
1864
1824
2732
2596
2348
1236
4206
1492
1492
2012
1368
1984
(53956, 100)
(53956, 2)
1444
1430
3922
1006
1754
1314
278
666
320
398
410
500
1022
834
360
1816
5188
2884
5808
4246
4440
6348
4750
3558
468
330
370
6432
378

3818
7608
8418
1774
940
1666
658
6680
1630
1144
1866
3036
2922
3700
1352
1822
2104
2878
5948
3336
3538
2024
1680
3536
266
8080
6962
1646
3786
1906
4128
3582
1300
3610
2892
2420
104
4976
952
2200
1488
3276
1054
4314
3558
1220
494
2360
2166
(70550, 100)
(70550, 2)
2766
1504
1386
2254
2454
1532
2962
2516
1052
2294
2664
1840
7304
3946
5588
2226
4060
812
1816
4164
980
1348
3082
4412
4412
2388
3778
3264
3282
7810
3492
2708
4930
4676
3574
1764
5836
790
6392
4866
3920
2910
1916
4596
916
4720
2430
2102
4136
940
(76255, 100)
(76255, 2)
1664
420
2384
2866
3632
1326
3848
1388
4244
3362
3428
362
1682
2672
684
4492
3912
3000
3992
4972
144
1526
5032
1748
1892
684
3158
3162
2710
2606
2732
1536
1536
9326
2294
4406
952
6974
3118
2764
82
4448
2282
6654
810
2186
7068
1820
1236
14356
(74295, 100)
(74295, 2)
4440
4212
2710
2918
2054
160
2380
8872
2582
1108
3772
2540
3076
1828
2186
3022
2510
4130
944
1698
2262
2080
2562
1634
1536
3286
446
2518
2922
2544
18006
1204
2096
1992
1600
2716
2674
3246
3048
3702
392


2298
6942
6162
4672
1654
4362
6362
4280
2412
4350
1934
1194
1716
3384
8370
430
3178
3030
2660
966
2684
886
1388
2404
1048
4734
(89783, 100)
(89783, 2)
3444
7660
4872
1162
2752
4804
404
3634
4248
4038
7278
1546
442
1244
2706
2200
9684
1596
1192
4316
7094
3202
7320
7276
1002
2182
1856
1962
2450
2598
4480
10524
4694
9434
7552
714
302
430
502
502
4912
3498
2056
7014
2670
2462
988
16704
2178
4640
(93710, 100)
(93710, 2)
146
1030
6178
3434
5266
5818
2670
2288
4184
1444
1820
1552
1882
1396
2380
266
1494
5444
472
1808
520
748
758
2516
2134
858
5886
3902
3704
4158
6962
606
3574
3578
4580
1886
896
4202
5364
3416
14244
3800
1950
8046
1418
1014
5374
1892
2778
1244
(73990, 100)
(73990, 2)
1648
4786
3690
1344
508
3662
2736
4818
2732
990
4674
2042
2092
2056
1950
2312
2284
4112
2786
3032
2728
4260
2292
1998
1446
458
1894
2010
2066
588
3862
1196
1202
2414
2144
6420
3704
3670
3058
3370
4098
1514
3496
2454
2524
4092
4872
6610
4732
2592
(68509, 100)
(68509, 2)
4246
1920
4434
3282
7568
7794
2844
1866
5358


1256
1148
3748
1502
4092
650
7104
3094
4122
1512
188
2768
1742
714
1550
968
1278
1018
2758
5008
2846
1948
3510
1242
4062
3736
3354
394
2272
800
5758
3720
1628
2742
1172
1490
1316
774
21318
2614
564
3282
3330
6950
4782
4302
1478
2268
2624
(70733, 100)
(70733, 2)
2916
1912
2196
4718
2076
4324
2534
724
2440
732
1174
1402
4020
4860
2340
2664
1420
1476
1296
5484
772
3408
3196
2778
3672
3276
3844
660
2714
7322
3688
3196
1942
986
590
4510
9184
2256
4868
2952
2666
2820
1792
4686
4720
16602
8098
4690
3290
6324
(83605, 100)
(83605, 2)
2584
3044
2584
3972
2578
1634
7072
1962
1260
2942
3346
1606
5110
2062
1740
3534
2322
4386
280
4048
3610
1120
806
3830
1776
2308
11232
578
718
2028
2274
2352
7474
3504
1558
1822
5470
2786
2232
6338
2410
3388
2958
3550
1938
4006
4100
1448
1260
10696
(76318, 100)
(76318, 2)
4568
842
1914
2270
8524
4514
3058
4448
2528
2326
2508
864
5176
1680
1806
1730
1658
2856
1192
4090
1884
396
4218
4368
6132
2318
3700
11076
1626
8860
2458
2456
4840
5102
3390
1860
3254
14654
3138
747

2160
1272
1996
4200
6882
3792
1710
4046
2114
6108
2574
5164
1976
2464
1748
6568
1076
902
5620
(79674, 100)
(79674, 2)
2970
1680
2226
1648
4816
2234
5016
2836
5790
2752
2918
1826
2028
3698
1588
1540
2632
2842
2842
1042
230
3738
5070
2426
1880
2526
2872
1520
1860
1888
2846
1812
4628
1780
1642
946
2782
15780
1456
466
1016
780
972
2586
1802
4230
3996
3432
2464
1668
(65509, 100)
(65509, 2)
3318
2988
2286
2084
3510
7190
840
4452
1928
3244
498
1596
1368
1686
2830
2526
3438
2028
4698
1024
4688
2770
9318
5402
2006
1642
4690
882
2048
1040
2308
3864
656
7094
3044
3348
2642
202
3176
1940
3220
1420
3096
13688
1358
4102
3224
4404
5166
2216
(76593, 100)
(76593, 2)
4284
3860
3404
2684
4132
1526
2772
2424
3676
1978
506
3384
3166
2446
1722
1486
1664
3168
2050
2412
4376
228
4578
1752
2806
4112
292
2618
1884
732
1146
2962
1068
1898
694
2430
7740
3972
1502
4128
3222
2438
6850
2110
5460
752
6442
2456
1972
2768
(66566, 100)
(66566, 2)
1446
1102
1078
6572
1090
5276
4232
2178
6780
2660
3570
3152
2062
10994
269

2248
3558
792
6926
2530
1738
3404
3344
6630
2290
106
1364
2520
5430
2968
2560
120
3994
3042
140
1998
2012
3476
6296
5172
(65693, 100)
(65693, 2)
3046
6782
3100
7700
9606
5384
3536
2592
10290
5082
3858
1472
474
252
1118
294
1142
2008
1198
830
482
3750
1198
6896
3856
9572
2290
1770
10072
6336
1974
1608
3036
1018
3090
5060
5110
1606
4650
1788
2216
2620
4118
6314
818
3002
2050
2976
2152
800
(83496, 100)
(83496, 2)
3622
1206
1044
1436
498
844
924
630
1068
2062
4540
1126
644
3288
352
4702
1408
1552
2868
2474
9406
3254
1440
1016
4448
5174
2420
2432
2562
4722
4842
2996
1940
988
830
3334
502
984
1370
1022
1870
536
1820
1316
640
232
274
400
392
202
(47326, 100)
(47326, 2)
1926
1356
2624
1950
2260
1176
1700
1654
2240
2228
3250
2576
2112
882
618
3530
1096
2546
3176
4090
5678
12708
2192
426
4004
208
2728
538
636
1112
3798
6610
6204
2972
178
2884
3334
2750
1990
1588
5424
2122
1070
1070
2710
1472
1840
5072
6318
1704
(64665, 100)
(64665, 2)
6478
1218
2654
818
2702
860
2482
2008
3416
4780
5112
2468
101

2030
4120
2376
654
12940
2250
778
498
2596
4646
3610
4566
5496
2842
3922
2282
1560
2980
1122
3972
804
3504
3254
5454
1926
2310
(81738, 100)
(81738, 2)
16114
3736
1088
3342
14088
1804
2974
7320
6026
1186
3158
12138
3514
998
4100
2942
1912
3238
974
7674
286
1224
1404
1048
2366
1378
1732
442
3522
3186
6834
2664
3098
3038
2030
4088
3562
6812
3396
984
1606
5116
1712
2190
836
5088
3840
3424
3420
1926
(87789, 100)
(87789, 2)
2198
2198
4264
1152
1012
3284
1720
3434
5694
5392
8024
6558
5204
406
5164
606
1650
1276
3316
2284
3078
1000
1110
1592
3494
408
218
1008
3564
2978
578
4564
2690
2072
1282
1448
928
3248
7880
1630
952
5092
6378
2302
1194
7992
11238
604
8904
3078
(76170, 100)
(76170, 2)
2730
3734
1816
1592
1326
2256
1192
908
762
2134
2030
838
1704
7050
364
2562
3964
1830
3368
398
5202
1666
1268
792
992
870
3538
4824
3074
710
3692
2348
4156
442
1334
2650
222
690
684
858
378
3116
3136
3882
1050
5112
1540
650
5826
3968
(53114, 100)
(53114, 2)
5644
7814
712
1950
6368
2478
1224
1816
11062
1006
144

In [9]:
np.save("test_values_x.npy", x)
np.save("test_values_y.npy", y)